In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# "Computs" center of "mass" for radial bins
def calc_rad(dat):
    # TODO Something is wrong here
    # starts at 0, didn't it start at 4?
    rad1,counts = np.unique(dat[:,0], return_counts=True)
    rad2 = np.unique(dat[:,1])
    rad = rad2-(rad2-rad1)/2.0
    counts = counts[0]
    return rad,counts
    

def Coord_Get(fl_in):
    '''
    Function to determine the radius and theta matrix
    plt_size is dependent on the size of the membrane plotted
    *** plt_size is no longer used remove ***
    
    dr, dth, theta, radius = Coord_Get(50)
    '''
    dat = np.loadtxt(fl_in, skiprows=1)
    Ntheta = np.shape(dat)[1]-4
    frames = 1
    #rad = dat[:,1]+(dat[:,1]-dat[:,0])/2.0    
    rad, frames = calc_rad(dat)
    dr = rad[1]-rad[0]
    the = np.linspace(0,2*np.pi,Ntheta)
    dth = the[1]-the[0]
    theta,radius=np.meshgrid(the,rad)

    return rad, dr, dth, theta, radius, frames, Ntheta

def get_header_info(fpath):
    '''
    Parameters
    ----------
    sat_files : str
        path to a data file. extracts extra information
        used for analysis

    Returns
    -------
    list
        Number of molecules, number of beads per mol, and expected
        density.
    '''
    line = open(fpath,'r').readline()
    num_mol = float(line.split(",")[0].split(":")[1].split(" ")[1])
    beads = float(line.split(",")[1].split(":")[1].split(" ")[1])
    avg_A = float(line.split(',')[2].split(":")[1].split(" ")[1])
    exrho = float(line.split(",")[3].split(":")[1].split("/")[0])
    avg_chain = float(line.split(",")[4].split(":")[1].split(" ")[1])
    return num_mol,avg_A,beads,exrho,avg_chain

In [ ]:
lipid = "POPG"
leaf = "upp"
data_root = Path("/Users/ezry/Projects/ELIC_PE_PG/trajectory_version/")
fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

In [ ]:
num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
the_data = np.loadtxt(fpath, skiprows=1)
rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)


In [ ]:
def get_shell(some_data, inner_radius, outer_radius, frames, Ntheta, dr):
    mask = np.logical_and(some_data[:,0] >= inner_radius, some_data[:,1] <= outer_radius)
    meta_shell = some_data[mask, 3:-1]
    nshells = (outer_radius-inner_radius)/dr
    assert nshells == int(nshells), "Error: non-integer number of shells"
    reshaped = np.reshape(meta_shell, (int(nshells),frames, Ntheta))
    flattened = np.sum(reshaped, axis=0)
    return flattened

In [ ]:
inner_r = 25
outer_r = 30
test_shell = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
site = test_shell[:,29]
frequencies = np.bincount(site.astype(int).flatten())
densities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(np.arange(len(frequencies)), densities)

def circle_area(r):
    return np.pi*r**2

Nexpected = exrho*(circle_area(outer_r)-circle_area(inner_r))/Ntheta
experimental_mean = np.mean(site)
ax.vlines(Nexpected, 0, np.max(densities), color='k', linestyles='dashed', label="expected mean")
ax.vlines(experimental_mean, 0, np.max(densities), color='red', linestyles='solid', label="actual mean")
ax.legend()
ax.set_xlabel(r"$N_\mathrm{beads}$")
ax.set_ylabel("Probability Density")
ax.set_title(f"One bin: {lipid} {leaf}er leaflet between {inner_r} and {outer_r} Angstroms from origin")
plt.savefig(f"one_bin_site_{lipid}_{leaf}.pdf")

In [ ]:
inner_r = 25
outer_r = 30
test_shell = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
site = np.mean(test_shell[:,5:30:6], axis=1)
frequencies = np.bincount(site.astype(int).flatten())
densities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(np.arange(len(frequencies)), densities)

def circle_area(r):
    return np.pi*r**2

Nexpected = exrho*(circle_area(outer_r)-circle_area(inner_r))/Ntheta
experimental_mean = np.mean(site)
ax.vlines(Nexpected, 0, np.max(densities), color='k', linestyles='dashed', label="expected mean")
ax.vlines(experimental_mean, 0, np.max(densities), color='red', linestyles='solid', label="actual mean")
ax.legend()
ax.set_xlabel(r"$N_\mathrm{beads}$")
ax.set_ylabel("Probability Density")
ax.set_title(f"One bin: {lipid} {leaf}er leaflet between {inner_r} and {outer_r} Angstroms from origin")
plt.savefig(f"one_bin_site_{lipid}_{leaf}.pdf")

In [ ]:
inner_r = 25
outer_r = 35
four_bins = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
site = np.sum(four_bins[:,28:30], axis=1)

frequencies = np.bincount(site.astype(int).flatten())
densities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(np.arange(len(frequencies)), densities)

def circle_area(r):
    return np.pi*r**2

area_four = 2*(circle_area(outer_r)-circle_area(inner_r))/Ntheta
Nexpected = (area_four)*exrho
experimental_mean = np.mean(site)
ax.vlines(Nexpected, 0, np.max(densities), color='k', linestyles='dashed', label="expected mean")
ax.vlines(experimental_mean, 0, np.max(densities), color='red', linestyles='solid', label="actual mean")
ax.legend()
ax.set_xlabel(r"$N_\mathrm{beads}$")
ax.set_ylabel("Probability Density")
ax.set_title(f"Four bins near the M4-M3 extracellular site")
plt.savefig(f"four_bin_site_{lipid}_{leaf}.pdf")

In [ ]:
inner_r = 25
outer_r = 35
four_bins = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
one_bin = get_shell(the_data, 20, 25, frames, Ntheta, dr)
site = np.sum(four_bins[:,28:30], axis=1)+one_bin[:,28]

frequencies = np.bincount(site.astype(int).flatten())
densities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(np.arange(len(frequencies)), densities)

def circle_area(r):
    return np.pi*r**2

area_four = 2*(circle_area(outer_r)-circle_area(inner_r))/Ntheta
area_one = (circle_area(25)-circle_area(20))/Ntheta
Nexpected = (area_four+area_one)*exrho
experimental_mean = np.mean(site)
ax.vlines(Nexpected, 0, np.max(densities), color='k', linestyles='dashed', label="expected mean")
ax.vlines(experimental_mean, 0, np.max(densities), color='red', linestyles='solid', label="actual mean")
ax.legend()
ax.set_xlabel(r"$N_\mathrm{beads}$")
ax.set_ylabel("Probability Density")
ax.set_title(f"Five bins near the M4-M3 extracellular site")
plt.savefig(f"five_bin_site_{lipid}_{leaf}.pdf")

In [ ]:
inner_r = 25
outer_r = 35
four_bins = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
one_bin = get_shell(the_data, 20, 25, frames, Ntheta, dr)
site = np.sum(four_bins[:,28:30], axis=1)+one_bin[:,28]

frequencies = np.bincount(site.astype(int).flatten())
densities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(np.arange(len(frequencies)), densities)

def circle_area(r):
    return np.pi*r**2

area_four = 2*(circle_area(outer_r)-circle_area(inner_r))/Ntheta
area_one = (circle_area(25)-circle_area(20))/Ntheta
Nexpected = (area_four+area_one)*exrho
experimental_mean = np.mean(site)
ax.vlines(Nexpected, 0, np.max(densities), color='k', linestyles='dashed', label="expected mean")
ax.vlines(experimental_mean, 0, np.max(densities), color='red', linestyles='solid', label="actual mean")
ax.legend()
ax.set_xlabel(r"$N_\mathrm{beads}$")
ax.set_ylabel("Probability Density")
ax.set_title(f"Five bins near the M4-M3 extracellular site")
plt.savefig(f"five_bin_site_{lipid}_{leaf}.pdf")